<a href="https://colab.research.google.com/github/m-wessler/nbm-verification/blob/main/nbm_textfile_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [113]:
import re
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

# File path
input_file_path = "/content/blend_nbptx.t13z.txt"

# Extract the base name of the input file
input_file_name = os.path.basename(input_file_path).split('.')[0]  # Remove extension for output filename

# Read the file
with open(input_file_path, "r") as file:
    lines = file.read().splitlines()

# Split the content into blocks based on blank rows
blocks = []
current_block = []
for line in lines:
    if not line.strip():  # Blank line indicates end of a block
        if current_block:
            blocks.append(current_block)
            current_block = []
    else:
        current_block.append(line)

# Add the last block if it wasn't added
if current_block:
    blocks.append(current_block)

# Process metadata from the first row of each block, discarding invalid blocks
valid_blocks = []
output_init_time = None  # Will store the init_time for the output filename
for block in blocks:
    if block:  # Ensure the block isn't empty
        metadata_line = block[0]  # First row contains metadata
        parts = re.split(r"\s{2,}", metadata_line.strip())  # Split by 2+ spaces

        # Validate metadata: ensure we have a proper init time (last two parts of the metadata line)
        if len(parts) >= 2:
            site_id = parts[0].split()[0]  # Extract SITE_ID (first part)
            init_time = " ".join(parts[-2:])  # Combine date and time (last two parts)

            # Check if the init_time is in the correct format
            if re.match(r"^\d{1,2}/\d{1,2}/\d{4} \d{4} UTC$", init_time):
                valid_blocks.append((site_id, init_time, block))  # Store valid blocks
                if output_init_time is None:
                    # Format init_time for the output filename (yyyymmddhh)
                    init_time_dt = datetime.strptime(init_time, "%m/%d/%Y %H%M %Z")
                    output_init_time = init_time_dt.strftime("%Y%m%d%H")
            else:
                print(f"Discarding block due to invalid init time: {metadata_line}")
        else:
            print(f"Discarding block due to insufficient metadata: {metadata_line}")

# Prepare the DataFrame
df = pd.DataFrame()

# Start parsing data rows for each valid block
for site_id, init_time, block in valid_blocks:
    print(f"Processing block for Site ID: {site_id}, Init Time: {init_time}")

    # Convert init_time to a datetime object
    init_time_dt = datetime.strptime(init_time, "%m/%d/%Y %H%M %Z")

    # Skip the first three rows (metadata + headers)
    data_rows = block[3:]

    # Initialize a dictionary to store the data for the current block
    block_data = {"site_id": site_id, "init_time": init_time_dt}

    # Parse each data row
    for row in data_rows:
        # Extract the variable name (first 5 characters after 1 space)
        variable_name = row[1:6].strip()

        # Parse the data columns
        parsed_data = [
            row[7:10].strip(), row[11:14].strip(), row[15:18].strip(), row[19:22].strip(),
            row[23:26].strip(), row[27:30].strip(), row[31:34].strip(), row[35:38].strip(),
            row[39:42].strip(), row[43:46].strip(), row[47:50].strip(), row[51:54].strip(),
            row[55:58].strip(), row[59:62].strip(), row[63:66].strip(), row[67:70].strip(),
            row[71:74].strip()
        ]

        # Replace blank values with np.nan
        parsed_data = [np.nan if val == "" else val for val in parsed_data]

        # Add the parsed data to the block_data dictionary
        block_data[variable_name] = parsed_data

    # Convert block_data to a DataFrame
    block_df = pd.DataFrame(block_data)

    # Calculate valid_time based on FHR
    if "FHR" in block_df.columns:
        block_df["FHR"] = pd.to_numeric(block_df["FHR"], errors="coerce")  # Ensure FHR is numeric
        block_df["valid_time"] = block_df["FHR"].apply(
            lambda x: init_time_dt + timedelta(hours=x) if not pd.isna(x) else np.nan
        )

    # Ensure numeric columns are properly set to int or float, filling NaN where necessary
    for column in block_df.columns:
        if column not in ["site_id", "init_time", "valid_time"]:  # Skip non-numeric columns
            block_df[column] = pd.to_numeric(block_df[column], errors="coerce")

    # Concatenate block DataFrame with the main DataFrame
    df = pd.concat([df, block_df], ignore_index=True)

# Set MultiIndex [init_time, valid_time, site_id]
df.set_index(["init_time", "valid_time", "site_id"], inplace=True)

# Generate the output filename
if output_init_time:
    output_file_name = f"{input_file_name}_{output_init_time}.csv"
else:
    output_file_name = f"{input_file_name}_output.csv"  # Fallback if no valid init_time is found
output_file_path = os.path.join("/content", output_file_name)

# Save the DataFrame to a CSV file
df.to_csv(output_file_path)
print(f"Data saved to {output_file_path}")

Discarding block due to insufficient metadata: 1
Processing block for Site ID: 086092, Init Time: 5/14/2024 1300 UTC
Processing block for Site ID: 188557, Init Time: 5/14/2024 1300 UTC
Processing block for Site ID: 220792, Init Time: 5/14/2024 1300 UTC
Processing block for Site ID: 2A1, Init Time: 5/14/2024 1300 UTC
Processing block for Site ID: 2M2, Init Time: 5/14/2024 1300 UTC
Processing block for Site ID: 356033, Init Time: 5/14/2024 1300 UTC
Processing block for Site ID: 359588, Init Time: 5/14/2024 1300 UTC
Processing block for Site ID: 382730, Init Time: 5/14/2024 1300 UTC
Processing block for Site ID: 41001, Init Time: 5/14/2024 1300 UTC
Processing block for Site ID: 41002, Init Time: 5/14/2024 1300 UTC
Processing block for Site ID: TROM6, Init Time: 5/14/2024 1300 UTC
Processing block for Site ID: TROM8, Init Time: 5/14/2024 1300 UTC
Processing block for Site ID: TRPG, Init Time: 5/14/2024 1300 UTC
Processing block for Site ID: TRPM, Init Time: 5/14/2024 1300 UTC
Processing bl

In [120]:
df.loc[(slice(None), slice(None), "VENU1")]

FHR  TXNMN  TXNSD  TXNP1  TXNP2  \
init_time           valid_time                                             
2024-05-14 13:00:00 2024-05-15 12:00:00   23   42.0    3.0   39.0   40.0   
                    2024-05-16 00:00:00   35   70.0    2.0   68.0   69.0   
                    2024-05-16 12:00:00   47   43.0    3.0   39.0   41.0   
                    2024-05-17 00:00:00   59   75.0    2.0   72.0   74.0   
                    2024-05-17 12:00:00   71   47.0    4.0   42.0   44.0   
                    2024-05-18 00:00:00   83   80.0    3.0   76.0   78.0   
                    2024-05-18 12:00:00   95   47.0    6.0   40.0   45.0   
                    2024-05-19 00:00:00  107   75.0    7.0   66.0   70.0   
                    2024-05-19 12:00:00  119   45.0    6.0   37.0   41.0   
                    2024-05-20 00:00:00  131   74.0    6.0   66.0   70.0   
                    2024-05-20 12:00:00  143   43.0    6.0   35.0   40.0   
                    2024-05-21 00:00:00  155   70.0    6.0   61.0   66.0   
                    2024-05-21 12:00:00  167   41.0    5.0   34.0   37.0   
                    2024-05-22 00:00:00  179   67.0    6.0   59.0   63.0   
                    2024-05-22 12:00:00  191   40.0    5.0   33.0   37.0   
                    2024-05-23 00:00:00  203   67.0    7.0   57.0   62.0   
                    2024-05-23 12:00:00  215   39.0    5.0   32.0   35.0   

                                         TXNP5  TXNP7  TXNP9  WSPP1  WSPP2  \
init_time           valid_time                                               
2024-05-14 13:00:00 2024-05-15 12:00:00   42.0   43.0   45.0      2      2   
                    2024-05-16 00:00:00   70.0   71.0   72.0     12     12   
                    2024-05-16 12:00:00   42.0   44.0   46.0      0      2   
                    2024-05-17 00:00:00   75.0   76.0   79.0     10     10   
                    2024-05-17 12:00:00   47.0   49.0   51.0      4      4   
                    2024-05-18 00:00:00   80.0   83.0   86.0      8     10   
                    2024-05-18 12:00:00   48.0   51.0   53.0      2      4   
                    2024-05-19 00:00:00   75.0   80.0   85.0      8     10   
                    2024-05-19 12:00:00   45.0   49.0   53.0      2      4   
                    2024-05-20 00:00:00   75.0   79.0   82.0      8     10   
                    2024-05-20 12:00:00   44.0   46.0   50.0      2      4   
                    2024-05-21 00:00:00   71.0   74.0   76.0      8     10   
                    2024-05-21 12:00:00   41.0   43.0   47.0      2      4   
                    2024-05-22 00:00:00   68.0   72.0   73.0      8     10   
                    2024-05-22 12:00:00   40.0   43.0   46.0      2      4   
                    2024-05-23 00:00:00   68.0   72.0   75.0      6      8   
                    2024-05-23 12:00:00   39.0   42.0   45.0      2      4   

                                         ...  I24P1  I24P2  I24P5  I24P7  \
init_time           valid_time           ...                               
2024-05-14 13:00:00 2024-05-15 12:00:00  ...    NaN    NaN    NaN    NaN   
                    2024-05-16 00:00:00  ...    NaN    NaN    NaN    NaN   
                    2024-05-16 12:00:00  ...    0.0    0.0    0.0    0.0   
                    2024-05-17 00:00:00  ...    NaN    NaN    NaN    NaN   
                    2024-05-17 12:00:00  ...    0.0    0.0    0.0    0.0   
                    2024-05-18 00:00:00  ...    NaN    NaN    NaN    NaN   
                    2024-05-18 12:00:00  ...    0.0    0.0    0.0    0.0   
                    2024-05-19 00:00:00  ...    NaN    NaN    NaN    NaN   
                    2024-05-19 12:00:00  ...    0.0    0.0    0.0    0.0   
                    2024-05-20 00:00:00  ...    NaN    NaN    NaN    NaN   
                    2024-05-20 12:00:00  ...    0.0    0.0    0.0    0.0   
                    2024-05-21 00:00:00  ...    NaN    NaN    NaN    NaN   
                    2024-05-21 12:00:00  ...    0